### Figure 1: Predicted images

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors

sns.set_style("white")
sns.set_palette(sns.color_palette("Paired"))
sns.set_palette(sns.color_palette("Set2"))

In [2]:
def regrid(array):
        hr_array = np.zeros((100,100))
        for i in range(10):
                for j in range(10):
                        i1 = i*10
                        i2 = (i+1)*10
                        j1 = j*10
                        j2 = (j+1)*10
                        hr_array[i1:i2,j1:j2] = array[i,j]
        return hr_array

In [3]:

plot = 'save'


# define datasets

real = np.load('/user/home/al18709/work/dsrnngan_predictions/validation_real-opt.npy')[0][:,:,:,0]
inputs = np.load('/user/home/al18709/work/dsrnngan_predictions/validation_input-opt.npy')[0][:,:,:,0]
pred_cnn = np.load('/user/home/al18709/work/cnn/unet_valid.npy')
pred_dsrnngan = np.load('/user/home/al18709/work/dsrnngan_predictions/validation_pred-opt.npy')[0][:,:,:,0]
pred_vaegan = np.load('/user/home/al18709/work/vaegan/predictions/validation_pred.npy')[0][:,:,:,0]

n = 5
m = 6
if plot == 'save':
		fig, axes = plt.subplots(n, m, figsize=(5*m, 5*n), sharey=True)
else:
		print('show')
		fig, axes = plt.subplots(n, m, figsize=(2*m, 2*n), sharey=True)

# make colours
range_ = (-5, 20)
storms = [32,70,20,60,70,80,90,99]
# axes[0,0].set_title('Real',size=24)
# axes[0,1].set_title('Predicted',size=24)
# axes[0,2].set_title('Input',size=24)
for i in range(m):
	j = 0
	storm = storms[i]
	print('storm max: ',real[storm].max())
	print('cnn max: ',pred_cnn[storm].max())
	print('vaegan max: ',pred_vaegan[storm].max())
	print('dsrnngan max: ',pred_dsrnngan[storm].max())
	print('inputs max: ', np.nanmax(inputs[storm]))
	axes[j,i].imshow(regrid(inputs[storm]), interpolation='nearest', norm=colors.Normalize(*range_), extent=None,cmap='Blues')
	axes[j+1,i].imshow(pred_cnn[storm], interpolation='nearest',norm=colors.Normalize(*range_), extent=None,cmap='Blues')
	axes[j+2,i].imshow(pred_vaegan[storm], interpolation='nearest',norm=colors.Normalize(*range_), extent=None,cmap='Blues')
	axes[j+3,i].imshow(pred_dsrnngan[storm], interpolation='nearest',norm=colors.Normalize(*range_), extent=None,cmap='Blues')
	axes[j+4,i].imshow(real[storm], interpolation='nearest',norm=colors.Normalize(*range_), extent=None,cmap='Blues')
	axes[j,i].set(xticklabels=[])
	axes[j,i].set(yticklabels=[])
	axes[j+1,i].set(xticklabels=[])
	axes[j+1,i].set(yticklabels=[])
	axes[j+2,i].set(xticklabels=[])
	axes[j+2,i].set(yticklabels=[])
	axes[j+3,i].set(xticklabels=[])
	axes[j+3,i].set(yticklabels=[])
	axes[j+4,i].set(xticklabels=[])
	axes[j+4,i].set(yticklabels=[])

fig.tight_layout()

if plot == 'save':
		plt.savefig('figure_1.png',bbox_inches='tight')
		plt.clf()
else:
		plt.show()

storm max:  33.6875
cnn max:  21.993977
vaegan max:  24.300207138061523
dsrnngan max:  32.2762336730957
inputs max:  17.709686
storm max:  28.875
cnn max:  19.708921
vaegan max:  32.625213623046875
dsrnngan max:  58.67399597167969
inputs max:  16.853546
storm max:  54.75
cnn max:  30.333626
vaegan max:  39.894187927246094
dsrnngan max:  44.54277420043945
inputs max:  23.184355
storm max:  40.75
cnn max:  22.252325
vaegan max:  32.40760040283203
dsrnngan max:  47.15876770019531
inputs max:  20.456804
storm max:  28.875
cnn max:  19.708921
vaegan max:  32.625213623046875
dsrnngan max:  58.67399597167969
inputs max:  16.853546
storm max:  40.0
cnn max:  19.630817
vaegan max:  37.4445686340332
dsrnngan max:  24.46809959411621
inputs max:  18.579762


<Figure size 2160x1800 with 0 Axes>